In [22]:
import time
from datetime import datetime
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from pyspark.sql.types import ArrayType, StringType, DateType, IntegerType, DoubleType
from pyspark.sql.functions import col, udf, monotonically_increasing_id, lag, datediff, collect_list, desc
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer, StopWordsRemover
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import SparseVector, DenseVector, Vectors, VectorUDT
from pyspark.sql.window import Window
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [23]:
query = '''
select text from orestist.conf_calls_sec 
where conf_calls_sec.section = 'man'
and conf_calls_sec.title = 'Q4 2003 Earnings Call'
limit 5

'''

df = sqlContext.sql(query)

In [27]:
# # numeric vector

# cv = CountVectorizer()\
#     .setInputCol("words")\
#     .setOutputCol("countVec")\
#     .setVocabSize(200)\
#     .setMinTF(1)\
#     .setMinDF(1)
# fittedCV = cv.fit(tokenized)
# fittedCV.transform(tokenized).show()
# prepped = cvfitted.transform(tokenized)

In [28]:
tkn = RegexTokenizer().setInputCol("text").setOutputCol("tokens")
tokenized = tkn.transform(df.select("text"))
tokenized.show(5)

+--------------------+--------------------+
|                text|              tokens|
+--------------------+--------------------+
|good day ladies a...|[good, day, ladie...|
|good morning and ...|[good, morning, a...|
|good morning ladi...|[good, morning, l...|
|good afternoon my...|[good, afternoon,...|
|good morning ladi...|[good, morning, l...|
+--------------------+--------------------+

In [29]:
englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops =  StopWordsRemover()\
    .setStopWords(englishStopWords)\
    .setInputCol("tokens") \
    .setOutputCol("words")
tokenized = stops.transform(tokenized.select("tokens"))

In [30]:
# from pyspark.ml.feature import NGram
# unigram = NGram().setInputCol("words").setN(1)
# bigram = NGram().setInputCol("words").setN(2)
# unigram.transform(tokenized.select("words")).show()
# bigram.transform(tokenized.select("words")).show()

In [31]:
#applying LDA - to imput text data into LDA convert to numeric format using countvector 

cv = CountVectorizer()\
    .setInputCol("words")\
    .setOutputCol("features")\
    .setVocabSize(200)\
    .setMinTF(1)\
    .setMinDF(1)\
    .setBinary(True)
cvfitted = cv.fit(tokenized)
prepped = cvfitted.transform(tokenized)



In [32]:
from pyspark.ml.clustering import LDA
lda = LDA().setK(5).setMaxIter(5) # K is the number of topics 
lda.explainParams()
model = lda.fit(prepped)


In [33]:
model.describeTopics(5).show()
vocab = cvfitted.vocabulary #vocab is a list

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[117, 78, 9, 26, ...|[0.00645118874944...|
|    1|[15, 175, 192, 63...|[0.00613633039669...|
|    2|[40, 77, 128, 192...|[0.00615479010200...|
|    3|[24, 135, 148, 9,...|[0.00610157778374...|
|    4|[146, 62, 64, 110...|[0.00619146191743...|
+-----+--------------------+--------------------+

In [37]:
# #Output topics. Each is a distribution over words (matching word count vectors)
# print("Learned topics (as distributions over vocab of " + str(model.vocabSize())
#       + " words):")
# topics = model.topicsMatrix().toArray() #this is an array

# #print(topics)

# for i in range(1,200):
#     print(vocab[i])
#     print(topics[i])
    

# for topic in range(5):
#     print("Topic " + str(topic) + ":")
    
#     #for word in range(0, 200):
#         #print(word)

In [38]:
def top_words_for_topic(topic_index, num_top_words=10):
    df_topic = pd.DataFrame.from_dict({
        'word': cvfitted.vocabulary,
        'weight': [topics[word_index][topic_index] for word_index in range(model.vocabSize())]
    })
    return df_topic.sort_values('weight', ascending=False).iloc[0:num_top_words+1]

In [39]:
#lda_model = LDA.train(corpus, k=num_topics)
num_topics = 5
num_top_words = 20
dfs_topics = []
for topic_index in range(num_topics):
    df_topic = top_words_for_topic(topic_index=topic_index, num_top_words=num_top_words)
    dfs_topics.append(df_topic.reset_index()[['word']]\
                              .rename(columns={'word': 'topic_' + str(topic_index + 1)}))
df_topics = pd.concat(dfs_topics, axis=1)
df_topics

         topic_1    topic_2    topic_3   topic_4      topic_5
0         global          5      lower  products         imrt
1   acquisitions   spending      level  estimate    netscreen
2       earnings  questions    looking        16         2002
3              2  september  questions  earnings          tax
4           june  increased       fire      cash         mtvr
5       compared   products    backlog      mtvr       demand
6       wireless      thank   wireless   welcome      quarter
7             30     demand    defense   believe        based
8        another       last      costs   average          per
9      estimates    systems      focus       one    operating
10     netscreen    results    forward    growth      looking
11       systems    netcool          0         2         june
12    government          1    diluted  oncology      segment
13        period     income     fourth  recovery  significant
14   improvement      major      forma    annual         fire
15      

In [16]:
# topics = model.describeTopics(maxTermsPerTopic = 15)
# for x, topic in enumerate(topics):
#     print ('topic nr: ' + str(x))
#     words = topic[0]
#     weights = topic[1]
#     for n in range(len(words)):
#         print (cvmodel.vocabulary[words[n]] + ' ' + str(weights[n]))